In [53]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.keras as kr
import os, sys

In [54]:
import utils

In [55]:
#Loading the data
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

train.shape, test.shape

((42000, 785), (28000, 784))

In [56]:
#Preparing the data
x_train = train.drop(["label"], axis=1).astype("float32")
y_train = train["label"].astype("int32")
x_test = test.astype("float32")
x_train = (x_train.values.reshape(-1, 28, 28, 1))/255.0
x_test = (x_test.values.reshape(-1, 28, 28, 1))/255.0
y_train = kr.utils.to_categorical(y_train, 10)

In [57]:
x_train.shape, x_test.shape, y_train.shape

((42000, 28, 28, 1), (28000, 28, 28, 1), (42000, 10))

In [69]:
model = kr.models.Sequential([
    kr.layers.Dense(521, activation="relu", input_shape=(28, 28, 1)),
    kr.layers.Dense(512, activation="relu"),
    kr.layers.Flatten(),
    kr.layers.Dense(10, activation="softmax")
])

model.summary()

optimizer = kr.optimizers.Adam(
    learning_rate= 0.001,
    name = "Adam"
)

model.compile(
    loss = "categorical_crossentropy",
    optimizer = optimizer,
    metrics = ["accuracy"]
)

Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_72 (Dense)             (None, 28, 28, 521)       1042      
_________________________________________________________________
dense_73 (Dense)             (None, 28, 28, 512)       267264    
_________________________________________________________________
flatten_9 (Flatten)          (None, 401408)            0         
_________________________________________________________________
dense_74 (Dense)             (None, 10)                4014090   
Total params: 4,282,396
Trainable params: 4,282,396
Non-trainable params: 0
_________________________________________________________________


In [70]:
print("Shapes:  ", y_train.shape, model.output_shape)
print("Model complexity:  ", model.count_params())

Shapes:   (42000, 10) (None, 10)
Model complexity:   4282396


In [71]:
model.fit(x = x_train, y = y_train, batch_size=50, epochs = 4)

Epoch 1/4
840/840 [==============================] - 947s 1s/step - loss: 0.3660 - accuracy: 0.8936
Epoch 2/4
840/840 [==============================] - 991s 1s/step - loss: 0.2985 - accuracy: 0.9153
Epoch 3/4
840/840 [==============================] - 946s 1s/step - loss: 0.2773 - accuracy: 0.9213
Epoch 4/4
840/840 [==============================] - 1006s 1s/step - loss: 0.2546 - accuracy: 0.9273


In [72]:
results = model.predict(x_test)
results = np.argmax(results, axis=1)
results = pd.Series(results, name="Label")
image_id = pd.Series(range(1, 28001), name = "ImageId")
submission = pd.concat([image_id, results], axis = 1)
submission.to_csv("../output/practice2.csv", index=False)